
## Merge CIVS data (6089)


In [1]:

## !pip install sktime


In [2]:

from sktime.forecasting.compose import EnsembleForecaster
from sklearn.svm import SVR
from sktime.transformations.series.detrend import ConditionalDeseasonalizer
from sktime.datasets import load_macroeconomic


In [3]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
## warnings.filterwarnings('ignore')
from statsmodels.tsa.api import seasonal_decompose, adfuller


In [4]:

import sktime
import statsmodels as sm
import matplotlib 
import sklearn

from sklearn.linear_model import( LinearRegression, 
                                 ElasticNet, 
                                 Ridge, 
                                 Lasso, 
                                 HuberRegressor)


from sktime.performance_metrics.forecasting import(MeanAbsolutePercentageError, 
                                                   MeanSquaredError, 
                                                   MeanAbsoluteScaledError)

from statsmodels.graphics.tsaplots import plot_acf


In [5]:

from sklearn.ensemble import (RandomForestRegressor, 
                              GradientBoostingRegressor, 
                              ExtraTreesRegressor)


from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor


In [6]:

from sktime.transformations.series.detrend import Deseasonalizer
from sktime.forecasting.base import ForecastingHorizon
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError as mape
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.model_selection import temporal_train_test_split

from sktime.transformations.series.detrend import Detrender
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.performance_metrics.forecasting import mean_squared_percentage_error as mspe
from sktime.performance_metrics.forecasting import mean_squared_error as mse
from sktime.performance_metrics.forecasting import mean_absolute_scaled_error as  mase


In [7]:

from sktime.forecasting.compose import make_reduction


In [8]:

from sktime.forecasting.model_selection import ForecastingGridSearchCV
## from sktime.split import                  SlidingWindowSplitter
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError


In [9]:

def split_data(df, test_split=0.15):
    n = int(len(df) * test_split)
    train, test = df[:-n], df[-n:]
    return train, test



## Read files to merge


In [10]:

data1 = pd.read_csv('CIVS_seq_real_6089.csv')
data2 = pd.read_csv('CIVS_seq_chem_1899.csv')


In [11]:

data1


,Date_f1,CNUM,LNUM_f1,SI_f1,S_f1,MN_f1,P_f1,CU_f1,NI_f1,CR_f1,MO_f1,TI_f1,HM_Temp_f1,ProbeSi_f1,Time_f1,Timestamp_f1,Compliant _eq_1_f1,C_perce_sat_f1,Before_After_f1,Good_bad_f1
0,44409,2808,161,0.900,0.058,0.463,0.069,0.003,0.017,0.047,0.002,0.024,2719,NaN,0.017361,44409.01736,1.0,4.825004,B,0.0
1,44409,2809,182,1.152,0.042,0.577,0.077,0.003,0.020,0.052,0.002,0.036,2677,NaN,0.057639,44409.05764,NaN,4.692156,B,0.0
2,44409,2809,171,0.932,0.053,0.521,0.071,0.003,0.015,0.051,0.002,0.027,2729,NaN,0.086111,44409.08611,1.0,4.830898,B,0.0
3,44409,2809,122,1.059,0.043,0.540,0.076,0.003,0.020,0.051,0.002,0.032,2736,NaN,0.131250,44409.13125,NaN,4.804690,B,0.0
4,44409,2810,147,0.917,0.059,0.478,0.072,0.003,0.017,0.049,0.002,0.024,2557,NaN,0.145833,44409.14583,1.0,4.587464,B,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6079,44592,380,138,0.547,0.027,0.347,0.052,0.003,0.003,0.038,0.002,0.038,2748,NaN,0.927778,44592.92778,1.0,4.990316,NaN,NaN
6080,44592,381,190,0.624,0.022,0.398,0.056,0.003,0.003,0.040,0.002,0.046,2753,NaN,0.010417,44592.01042,1.0,4.976264,NaN,NaN
6081,44592,381,132,0.552,0.025,0.363,0.054,0.003,0.003,0.039,0.002,0.040,2744,NaN,0.970833,44592.97083,1.0,4.984214,NaN,NaN
6082,44592,381,139,0.655,0.023,0.381,0.056,0.003,0.003,0.040,0.002,0.047,2759,NaN,0.998611,44592.99861,1.0,4.973488,NaN,NaN


In [12]:

 CNUM_LIST_DATA1 = data1['CNUM'].tolist()


In [13]:

data2


,date_f2,CNUM,Slag_Time_f2,Open_Time_f2,Close_Time_f2,Taphole_f2,Pipe_burn_hole_f2,Taphole_Length_f2,Clay_box_f2,Clay_Close_hole_ft3_f2,...,Cast_Avg_Ti_f2,Slag_Fe_f2,Slag_SiO2_f2,Slag_Al2O3_f2,Slag_CaO_f2,Slag_MgO_f2,Slag_S_f2,Slag_K2O_f2,Slag_B_SiO2_f2,Slag_TiO2_f2
0,8/1/2021 0:00,2809,00:59,00:55,03:23,2,0,11.4,602,2.75,...,0.032,0.05,38.709999,7.37,37.610001,10.59,1.06,0.38,1.25,0.41
1,8/1/2021 0:00,2810,03:45,03:20,06:01,3,0,11.5,596,4.00,...,0.024,0.05,36.020000,6.70,37.209999,9.71,1.15,0.30,1.30,0.34
2,8/1/2021 0:00,2811,06:00,05:55,08:29,2,0,11.9,602,2.25,...,0.025,0.05,38.689999,7.14,38.590000,10.74,1.08,0.31,1.27,0.40
3,8/1/2021 0:00,2812,08:45,08:25,11:04,3,0,11.9,596,3.50,...,0.030,0.05,38.380001,7.33,38.610001,10.83,1.26,0.31,1.29,0.35
4,8/1/2021 0:00,2813,11:10,11:03,13:18,2,0,10.7,646,2.50,...,0.025,0.06,38.419998,6.83,38.189999,10.65,1.14,0.31,1.27,0.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1892,1/31/2022 0:00,376,13:25,13:00,14:50,2,0,10.5,6,3.00,...,0.046,0.13,39.330002,8.57,41.480000,8.23,0.99,0.58,1.26,0.42
1893,1/31/2022 0:00,377,15:26,14:50,16:44,1,0,10.9,59,5.00,...,0.041,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00
1894,1/31/2022 0:00,378,16:55,16:41,19:14,2,0,10.5,6,3.00,...,0.057,1.68,38.900002,8.95,41.880001,8.32,1.10,0.44,1.29,0.38
1895,1/31/2022 0:00,379,19:20,19:12,20:54,1,0,9.6,59,5.00,...,0.052,0.51,39.639999,8.66,41.150002,8.27,0.93,0.63,1.25,0.44


In [14]:

 CNUM_LIST_DATA2 = data2['CNUM'].tolist()


In [15]:

len( set(CNUM_LIST_DATA1) )


1860

In [16]:

len( set(CNUM_LIST_DATA2) )


1897

In [17]:

 
unique_res = set(CNUM_LIST_DATA1) - set(CNUM_LIST_DATA2)
unique_res


{84,
 110,
 123,
 135,
 146,
 159,
 172,
 183,
 246,
 260,
 273,
 381,
 2808,
 3985,
 3999,
 4030,
 4036,
 4062,
 4075,
 4100,
 4111,
 4200,
 4212,
 4223,
 4296,
 4309}

In [18]:

len(unique_res)


26

In [19]:

## list( set(CNUM_LIST_DATA1) )


In [20]:

## list( set(CNUM_LIST_DATA2) )


In [21]:

merged_data = pd.merge(data1, data2, on='CNUM', how='inner')


In [22]:

merged_data


,Date_f1,CNUM,LNUM_f1,SI_f1,S_f1,MN_f1,P_f1,CU_f1,NI_f1,CR_f1,...,Cast_Avg_Ti_f2,Slag_Fe_f2,Slag_SiO2_f2,Slag_Al2O3_f2,Slag_CaO_f2,Slag_MgO_f2,Slag_S_f2,Slag_K2O_f2,Slag_B_SiO2_f2,Slag_TiO2_f2
0,44409,2809,182,1.152,0.042,0.577,0.077,0.003,0.020,0.052,...,0.032,0.05,38.709999,7.37,37.610001,10.59,1.06,0.38,1.25,0.41
1,44409,2809,171,0.932,0.053,0.521,0.071,0.003,0.015,0.051,...,0.032,0.05,38.709999,7.37,37.610001,10.59,1.06,0.38,1.25,0.41
2,44409,2809,122,1.059,0.043,0.540,0.076,0.003,0.020,0.051,...,0.032,0.05,38.709999,7.37,37.610001,10.59,1.06,0.38,1.25,0.41
3,44409,2810,147,0.917,0.059,0.478,0.072,0.003,0.017,0.049,...,0.024,0.05,36.020000,6.70,37.209999,9.71,1.15,0.30,1.30,0.34
4,44409,2810,190,1.023,0.054,0.508,0.072,0.003,0.012,0.048,...,0.024,0.05,36.020000,6.70,37.209999,9.71,1.15,0.30,1.30,0.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5993,44592,379,120,0.795,0.019,0.397,0.054,0.003,0.003,0.041,...,0.052,0.51,39.639999,8.66,41.150002,8.27,0.93,0.63,1.25,0.44
5994,44592,379,121,NaN,NaN,0.000,0.000,0.000,0.000,0.000,...,0.052,0.51,39.639999,8.66,41.150002,8.27,0.93,0.63,1.25,0.44
5995,44592,380,163,0.598,0.025,0.383,0.054,0.003,0.003,0.039,...,0.041,0.32,39.090000,8.67,40.560001,7.86,0.99,0.56,1.24,0.42
5996,44592,380,146,0.608,0.030,0.351,0.052,0.003,0.003,0.038,...,0.041,0.32,39.090000,8.67,40.560001,7.86,0.99,0.56,1.24,0.42


In [23]:

merged_data.to_csv('merged_seq_5998_CIVS.csv')
